# AWS Lambda Operations - Comprehensive Guide

This notebook provides a complete walkthrough of AWS Lambda serverless compute operations using boto3 SDK.

## Prerequisites - Manual AWS Setup

Before running this notebook, the following setup was completed manually in AWS Console:

### Step 1: Create IAM Role for Lambda Execution
- Logged into AWS account
- Navigate to **IAM** > **Roles** > **Create role**
- Select trusted entity: **AWS service** > **Lambda**
- Attach policies:
  - **AWSLambdaBasicExecutionRole** (CloudWatch Logs access)
  - **AmazonS3FullAccess** (for S3 integration examples)
- Role name: **lambda-execution-role**
- Copy the Role ARN (needed for creating functions)

### Step 2: IAM User for Lambda Management
- Created IAM user with **programmatic access**
- Attached policies:
  - **AWSLambda_FullAccess** (manage Lambda functions)
  - **IAMReadOnlyAccess** (read roles for function creation)
- Generated access credentials (Access Key ID, Secret Access Key)
- Saved credentials securely in `.env` file

### Step 3: Create Simple Lambda Function (Manual Setup)
- Navigate to **Lambda** > **Create function**
- Function name: **hello-world-function**
- Runtime: **Python 3.12**
- Execution role: Use existing role **lambda-execution-role**
- Created function successfully
- This will be used for testing our boto3 operations

### Step 4: Environment File
Created `.env` file with the following variables:
```
AWS_ACCESS_KEY=your-access-key-id
AWS_SECRET_KEY=your-secret-access-key
AWS_REGION=us-east-2
LAMBDA_ROLE_ARN=arn:aws:iam::YOUR-ACCOUNT-ID:role/lambda-execution-role
```

### Step 5: Python Environment
- Installed boto3 library for AWS SDK
- Installed python-dotenv for secure credential loading

Now we're ready to explore Lambda operations programmatically.

---

## What You'll Learn

### Phase 1: Basic Lambda Operations (Read-Only)
- List all Lambda functions in your account
- Get function details (configuration, memory, timeout)
- Invoke functions and retrieve responses
- View function logs

### Phase 2: Function Management (Create & Deploy)
- Create Lambda functions programmatically
- Upload and update function code
- Delete functions
- Package Python code for deployment

### Phase 3: Configuration & Environment
- Set environment variables
- Configure memory and timeout settings
- Update execution role
- Manage function aliases and versions

### Phase 4: Event Sources & Triggers
- Create S3 event triggers
- Set up scheduled execution (EventBridge/CloudWatch)
- Configure API Gateway integration
- Process event data from different sources

### Phase 5: Advanced Features
- Create and attach Lambda layers
- View CloudWatch logs programmatically
- Implement error handling and retries
- Monitor function metrics
- Optimize performance and cost

## Lambda Architecture Overview

```
┌─────────────────────────────────────────────────────────┐
│  AWS LAMBDA ARCHITECTURE                                │
│                                                         │
│  Event Sources                Lambda Function           │
│  ┌──────────────┐            ┌──────────────┐          │
│  │ S3 Upload    │───────────>│              │          │
│  │ API Request  │───────────>│  Your Code   │          │
│  │ Schedule     │───────────>│  (Handler)   │          │
│  │ Manual Invoke│───────────>│              │          │
│  └──────────────┘            └──────┬───────┘          │
│                                     │                  │
│                                     ▼                  │
│  AWS Manages:              ┌──────────────┐            │
│  • Server provisioning     │   Outputs:   │            │
│  • Scaling                 │   • Return   │            │
│  • High availability       │   • S3 Write │            │
│  • Patching               │   • Database │            │
│  • Monitoring             └──────────────┘            │
│                                                         │
│  You Pay Only For:                                      │
│  • Number of requests (invocations)                     │
│  • Compute time (GB-seconds)                            │
└─────────────────────────────────────────────────────────┘
```

## Key Concepts

**Function**: Your code package with handler, runtime, and dependencies

**Handler**: Entry point function (e.g., `lambda_handler(event, context)`)

**Runtime**: Execution environment (Python 3.12, Node.js 20, Java 11, etc.)

**Event**: Input data passed to your function

**Context**: Runtime information (request ID, memory limit, etc.)

**Execution Role**: IAM role that grants permissions to your function

**Trigger**: Event source that invokes your function

**Cold Start**: First invocation after deployment (slower)

**Warm Start**: Subsequent invocations (faster, reuses container)